In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False
%matplotlib notebook

In [2]:
import pandas as pd
from fuzzywuzzy import process
import re
import json
from collections import defaultdict
import requests
import io
import os
from google.cloud import storage

In [3]:
from voting_bot_utils import get_master_issue_json_from_cloud, prepare_master_issues_json, PROJECT_BUCKET_REF
from voting_bot_utils import put_data_to_gcp_bucket_file, update_webpage

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r'uk-voting-bot-2c521de46d41.json'
os.environ[PROJECT_BUCKET_REF]=r'uk-voting-bot-cloud-bucket-110922'

### Get new data

In [37]:
divisionNumber=1287
# csv_link = f'https://votes.parliament.uk/Divisions/DownloadCSV/{divisionNumber}?house=Commons'
vote_url = f'https://votes.parliament.uk/Votes/Commons/Division/{divisionNumber}'
vote_url

'https://votes.parliament.uk/Votes/Commons/Division/1287'

In [40]:


downloaded_file = 'rwanda_Division1287.csv'


vote_data_retrieved = '2022-12-15'
working_title = 'Rwanda'
description = 'MPs vote to compel the government to inform and seek approval from parliament about removing asylum seekers to a third party.'
keywords = ['rwanda']

In [41]:
vote_data = pd.read_csv(f'reference_files/{downloaded_file}',skiprows=10)

### Check key words

In [42]:
master_issues = get_master_issue_json_from_cloud()


In [43]:
master_issues

[{'subject': 'Sewage',
  'keywords': ['sewage',
   'river shit',
   'beach shit',
   'environment',
   'amendment 45'],
  'vote_url': 'https://votes.parliament.uk/Votes/Commons/Division/1116',
  'description': 'MPs vote to allow the discharge of raw sewage into rivers and seas.',
  'filename': 'Sewage_vote.json'},
 {'subject': 'Brexit',
  'keywords': ['brexit', 'eu withdrawl'],
  'vote_url': 'https://votes.parliament.uk/Votes/Commons/Division/749',
  'description': 'MPs vote on the 3rd reading of the Brexit vote.',
  'filename': 'Brexit_vote.json'},
 {'subject': 'Paterson',
  'keywords': ['paterson', 'standards'],
  'filename': 'Paterson_vote.json',
  'vote_url': 'https://votes.parliament.uk/Votes/Commons/Division/1124',
  'description': 'MPs vote not to suspend Owen Paterson after he was found guilty of paid lobbying by a standards watchdog. The amendment also provided for changing the system for investigating breaches of standards.'},
 {'subject': 'Fracking',
  'keywords': ['fracking

In [44]:
master_issue_dict, all_issues, issues_filenames = prepare_master_issues_json(master_issues)

In [45]:
for new_word in keywords:
    for old_word in all_issues:
        score = process.extract(new_word, [old_word], limit=5)
#         print(f'{old_word} - {new_word}: {score[0][1]}' )
        if score[0][1] >= 80:
#             print(f'{old_word} - {new_word}: {score[0][1]}' )
            print('##### Clash #####')

In [46]:
for new_word in keywords:
    for old_word in all_issues:
        score = process.extract(new_word, [old_word], limit=5)
        print(f'{old_word} - {new_word}: {score[0][1]}' )
        if score[0][1] >= 80:
#             print(f'{old_word} - {new_word}: {score[0][1]}' )
            print('##### Clash #####')

sewage - rwanda: 33
river shit - rwanda: 15
beach shit - rwanda: 15
environment - rwanda: 30
amendment 45 - rwanda: 45
brexit - rwanda: 17
eu withdrawl - rwanda: 33
paterson - rwanda: 29
standards - rwanda: 60
fracking - rwanda: 43
shale - rwanda: 18
school meals - rwanda: 15
rashford - rwanda: 43
gullis - rwanda: 0
asylum - rwanda: 17
echr - rwanda: 20


### Process Issue Data File

In [47]:
new_cols = [x.replace(' ','_') for x in vote_data.columns]
vote_data.columns=new_cols

In [48]:
vote_data['lower_name'] = vote_data.Member.str.lower()

In [49]:
vote_data.loc[vote_data.lower_name=='elizabeth truss','lower_name'] = 'liz truss'

In [50]:
vote_records = vote_data[['Member' ,'Party' ,'Constituency' ,'Vote' , 'lower_name']].to_dict(orient='records')

In [51]:
vote_records = { x['lower_name']:x for x in vote_records }

In [52]:
f_name = f'{working_title}_vote.json'
f_name

'Rwanda_vote.json'

In [53]:
obj={}
with open('formatted_vote_data/'+f_name,'w') as f:
    
    obj['vote_url']=vote_url
    obj['vote_data_retrieved']=vote_data_retrieved
    obj['title'] = working_title
    obj['keywords']=keywords
    obj['description']=description
    obj['vote_data'] = vote_records
    
    json.dump(obj,f,indent=2)

In [54]:
put_data_to_gcp_bucket_file(data=obj, filename=f_name)

### Process Master Data file

In [55]:
new_element = {'subject': working_title,
  'keywords': keywords,
  'filename': f_name,
  'vote_url':vote_url,   
  'description':description,  
              }

In [56]:
master_issues.append(new_element)

In [57]:
master_issues

[{'subject': 'Sewage',
  'keywords': ['sewage',
   'river shit',
   'beach shit',
   'environment',
   'amendment 45'],
  'vote_url': 'https://votes.parliament.uk/Votes/Commons/Division/1116',
  'description': 'MPs vote to allow the discharge of raw sewage into rivers and seas.',
  'filename': 'Sewage_vote.json'},
 {'subject': 'Brexit',
  'keywords': ['brexit', 'eu withdrawl'],
  'vote_url': 'https://votes.parliament.uk/Votes/Commons/Division/749',
  'description': 'MPs vote on the 3rd reading of the Brexit vote.',
  'filename': 'Brexit_vote.json'},
 {'subject': 'Paterson',
  'keywords': ['paterson', 'standards'],
  'filename': 'Paterson_vote.json',
  'vote_url': 'https://votes.parliament.uk/Votes/Commons/Division/1124',
  'description': 'MPs vote not to suspend Owen Paterson after he was found guilty of paid lobbying by a standards watchdog. The amendment also provided for changing the system for investigating breaches of standards.'},
 {'subject': 'Fracking',
  'keywords': ['fracking

In [58]:
## upload to bucket
put_data_to_gcp_bucket_file(data=master_issues,filename='master_issues_list.json')

### update the website

In [59]:
with open('website/template.html', 'r') as f:
    website_text = f.read()

In [60]:
table_text=''
for issue in master_issues:
    table_text+='<tr>'
    table_text += '<td>'+issue['subject']+'</td>'
    table_text += '<td>'+issue['description']+'</td>'
    table_text += '<td>'+', '.join(issue['keywords'])+'</td>'
    table_text += '<td><a href="'+issue['vote_url']+'">Link</a></td>'
    table_text+='</tr>'    

In [61]:
new_page = website_text.replace('MARKER',table_text)

In [62]:
# write locally
with open('website/main.html','w') as f:
    f.write(new_page)

In [63]:
update_webpage(new_page)

### send a tweet

In [64]:
import tweepy
from config import API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_SECRET

In [65]:
keyword_str = ', '.join(keywords)
tweet = f'New vote data added: {description} Keywords: {keyword_str}'
tweet

'New vote data added: MPs vote to compel the government to inform and seek approval from parliament about removing asylum seekers to a third party. Keywords: rwanda'

In [66]:
assert len(tweet) < 280

In [67]:
auth = tweepy.OAuth1UserHandler(
   API_KEY, API_SECRET,
   ACCESS_TOKEN , ACCESS_SECRET
)
api = tweepy.API(auth)

In [68]:
try:
    api.verify_credentials()
    print("Authentication Successful")
except:
    print("Authentication Error")

Authentication Successful


In [69]:
status = api.update_status(status=tweet)
status

Status(_api=<tweepy.api.API object at 0x7e196cf73a00>, _json={'created_at': 'Thu Dec 15 01:58:24 +0000 2022', 'id': 1603207816189296642, 'id_str': '1603207816189296642', 'text': 'New vote data added: MPs vote to compel the government to inform and seek approval from parliament about removing a… https://t.co/OBCrTlWz2p', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/OBCrTlWz2p', 'expanded_url': 'https://twitter.com/i/web/status/1603207816189296642', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'source': '<a href="http://www.jjcasswell.com" rel="nofollow">howdidtheyvoteuk</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1561845537950801921, 'id_str': '1561845537950801921', 'name': 'howdidtheyvoteuk', 'screen_name': 'howdidtheyvote1', 'location': 'UK', 'descriptio